In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
from sklearn.externals import joblib
import MySQLdb as mdb

import sys 
sys.path.append('..')
from algo import generateHeader

In [2]:
con = mdb.connect(host="127.0.0.1", port=3306, user="root", passwd="bdd", db="canvas_fp_project")
cur = con.cursor(mdb.cursors.DictCursor)

In [ ]:
def process_results_replay(folder, prefix_file):
    file_names = os.listdir("../save_res_old/"+folder)
    nb_days_to_tracking_time = dict()
    nb_days_to_nb_ids_assigned = dict()
    nb_days_to_nb_fps = dict()
    nb_days_to_max_chain_ratio = dict()
    nb_days_to_ownership = dict()
    nb_days_to_length_chain = dict()
    nb_days_to_nb_raw_days = dict()
    nb_days_to_acc = dict()
    nb_days_to_nb_ids_user = dict()
    for file_name in file_names:
        try:
            nb_days = file_name.split("_")[1]
            # special case for 0.5
            if "." in nb_days[:3]:
                nb_days = float("".join(re.findall('\d+', nb_days[:3])))/10.0
            else:
                nb_days = float("".join(re.findall('\d+', nb_days[:3])))
        except:
            nb_days = file_name.split("_")[2]
            nb_days = float("".join(re.findall('\d+', nb_days[:3])))
        
        if prefix_file in file_name:
            df = pd.read_csv("../save_res_old/"+ folder +"/%s" % file_name)
            if "res1" in file_name:
                # we recompute ratio so that the first fingerprint of each assigned id is not counted
                df["ratio"] = (df["nb_original_fp"] - df["nb_assigned_ids"])/df["nb_assigned_ids"]
                df["nb_raw_days"] = df["ratio"] * nb_days
                df["nb_days"] = df["ratio"]/df["nb_original_fp"]
                df["max_chain_ratio"] = (df["max_chain"]-1)*nb_days
                df["acc"] = 1.0/df["nb_assigned_ids"]
                nb_days_to_tracking_time[nb_days] = np.mean(df["nb_days"])
                nb_days_to_max_chain_ratio[nb_days] = np.mean(df["max_chain_ratio"])
                nb_days_to_nb_raw_days[nb_days] = np.mean(df["nb_raw_days"])
                nb_days_to_acc[nb_days] = np.mean(df["acc"])
                nb_days_to_nb_ids_user[nb_days] = np.mean(df["nb_assigned_ids"])
            elif "res2" in file_name:
                nb_days_to_nb_ids_assigned[nb_days] = np.mean(df["nb_assigned_ids"])
                nb_days_to_ownership[nb_days] = np.mean(df["ownership"])
                nb_days_to_length_chain[nb_days] = np.mean(df["nb_fingerprints"])
            elif "replay" in file_name:
                nb_days_to_nb_fps[nb_days] = len(df)
    
    pandas_dict = {
        "nb_days": [],
        "nb_raw_days": [],
        "tracking_time": [],
        "nb_ids_assigned": [],
        "nb_fps": [],
        "max_chain_ratio": [],
        "length_chain": [],
        "ownership": [],
        "acc": [],
        "nb_days_to_nb_ids_user": []
        
    }
    
    for nb_days in nb_days_to_tracking_time:
        pandas_dict["nb_days"].append(nb_days)
        pandas_dict["tracking_time"].append(nb_days_to_tracking_time[nb_days])
        pandas_dict["nb_raw_days"].append(nb_days_to_nb_raw_days[nb_days])
        pandas_dict["nb_ids_assigned"].append(nb_days_to_nb_ids_assigned[nb_days])
        pandas_dict["nb_fps"].append(nb_days_to_nb_fps[nb_days])
        pandas_dict["max_chain_ratio"].append(nb_days_to_max_chain_ratio[nb_days])
        pandas_dict["length_chain"].append(nb_days_to_length_chain[nb_days])
        pandas_dict["ownership"].append(nb_days_to_ownership[nb_days])
        pandas_dict["acc"].append(nb_days_to_acc[nb_days])
        pandas_dict["nb_days_to_nb_ids_user"].append(nb_days_to_nb_ids_user[nb_days])
#         print(nb_days, nb_days_to_tracking_time[nb_days], nb_days_to_nb_ids_assigned[nb_days],
#               nb_days_to_nb_fps[nb_days], nb_days_to_max_chain_ratio[nb_days],
#               nb_days_to_length_chain[nb_days], nb_days_to_ownership[nb_days])
    return pd.DataFrame(pandas_dict).sort_values("nb_days")
        

avg_presence_time = 109 #days
# replace with eckersley
df_eck = process_results_replay("final_results2", "eck")
# df_eck = process_results_replay("nov9", "eck")

x_eck = df_eck["nb_days"]
y_eck = df_eck["nb_raw_days"]


df_rule = process_results_replay("final_results2", "rule")
# df_rule = process_results_replay("nov9", "rule")

x_rule = df_rule["nb_days"]
y_rule = df_rule["nb_raw_days"]

# df_ml = process_results_replay("nov9", "newml2")
df_ml = process_results_replay("results_ml_new", "mlpar7diff")
x_ml = df_ml["nb_days"]
y_ml = df_ml["nb_raw_days"]

line1_eck, = plt.plot(x_eck, y_eck, label="Panopticlick")
line1_rules, = plt.plot(x_rule, y_rule, label="Rule-based")
line1_ml, = plt.plot(x_ml, y_ml, label="Hybrid")
plt.ylabel('Average tracking duration\n (days)', fontsize=15)
plt.xlabel('Collect frequency (days)', fontsize=15)
plt.legend(handles=[line1_eck, line1_rules, line1_ml], loc=0, fontsize=12)
plt.savefig('./raw_days_frequency.pdf', bbox_inches = 'tight')
plt.show()
plt.show()

y_eck_max_time = df_eck["max_chain_ratio"]
y_rule_max_time = df_rule["max_chain_ratio"]
y_ml_max_time = df_ml["max_chain_ratio"]

line2_eck, = plt.plot(x_eck, y_eck_max_time, label="Panopticlick")
line2_rules, = plt.plot(x_rule, y_rule_max_time, label="Rule-based")
line2_ml, = plt.plot(x_ml, y_ml_max_time, label="Hybrid")
plt.legend(handles=[line2_eck, line2_rules, line2_ml], loc=0, fontsize=12)
plt.ylabel('Average maximum tracking\n duration (days)', fontsize=15)
plt.xlabel('Collect frequency (days)', fontsize=15)

plt.savefig('./raw_max_days_frequency.pdf', bbox_inches = 'tight')
plt.show()


y_eck_ids = df_eck["nb_days_to_nb_ids_user"]
y_rule_ids = df_rule["nb_days_to_nb_ids_user"]
y_ml_ids = df_ml["nb_days_to_nb_ids_user"]

line3_eck, = plt.plot(x_eck, y_eck_ids, label="Panopticlick")
line3_rule, = plt.plot(x_rule, y_rule_ids, label="Rule-based")
line3_ml, = plt.plot(x_ml, y_ml_ids, label="Hybrid")
plt.legend(handles=[line3_eck, line3_rule, line3_ml], loc=0, fontsize=12)
plt.ylabel('Number of ids per user\n(lower is better)', fontsize=15)
plt.xlabel('Collect frequency (days)', fontsize=15)
plt.savefig('./nb_ids_frequency.pdf', bbox_inches = 'tight')
plt.show()


y_eck_own = df_eck["ownership"]
y_rule_own = df_rule["ownership"]
y_ml_own = df_ml["ownership"]

line4_eck, = plt.plot(x_eck, y_eck_own, label="Panopticlick")
line4_rule, = plt.plot(x_rule, y_rule_own, label="Rule-based")
line4_ml, = plt.plot(x_ml, y_ml_own, label="Hybrid")
plt.ylabel('Average ownership\n(higher is better)', fontsize=15)
plt.xlabel('Collect frequency (days)', fontsize=15)
plt.legend(handles=[line4_eck, line4_rule, line4_ml], loc=0, fontsize=12)
plt.savefig('./ownership_frequency.pdf', bbox_inches = 'tight')
plt.show()




# process_results_replay("ip")